In [2]:
import pandas as pd
import numpy as np

### 6.5 Lab 1: Subset Selection Methods
#### 6.5.1 Best Subset Selection
Here we apply the best subset selection approach to the Hitters data. We
wish to predict a baseball player’s Salary on the basis of various statistics
associated with performance in the previous year.
First of all, we note that the Salary variable is missing for some of the players.

In [26]:
# Load dataset
# We later want to fit using horsepower, so remove its rows with NA
hitters = pd.read_csv('Data/Hitters.csv', index_col = 0)
display(f"Missing data in {hitters.columns[hitters.isna().any()].tolist()}")
num_missing = np.sum(hitters.Salary.isnull())
print('We are missing Salary data for', num_missing, 'players.')
# Drop these players
hitters = hitters.dropna()
display(hitters.describe(include = "O"))
display(hitters.describe(include = [np.number]))

"Missing data in ['Salary']"

We are missing data for 59 players.


,League,Division,NewLeague
count,263,263,263
unique,2,2,2
top,A,W,A
freq,139,134,141


,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary
count,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000,263.000000
mean,403.642586,107.828897,11.619772,54.745247,51.486692,41.114068,7.311787,2657.543726,722.186312,69.239544,361.220532,330.418251,260.266160,290.711027,118.760456,8.593156,535.925882
std,147.307209,45.125326,8.757108,25.539816,25.882714,21.718056,4.793616,2286.582929,648.199644,82.197581,331.198571,323.367668,264.055868,279.934575,145.080577,6.606574,451.118681
min,19.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,19.000000,4.000000,0.000000,2.000000,3.000000,1.000000,0.000000,0.000000,0.000000,67.500000
25%,282.500000,71.500000,5.000000,33.500000,30.000000,23.000000,4.000000,842.500000,212.000000,15.000000,105.500000,95.000000,71.000000,113.500000,8.000000,3.000000,190.000000
50%,413.000000,103.000000,9.000000,52.000000,47.000000,37.000000,6.000000,1931.000000,516.000000,40.000000,250.000000,230.000000,174.000000,224.000000,45.000000,7.000000,425.000000
75%,526.000000,141.500000,18.000000,73.000000,71.000000,57.000000,10.000000,3890.500000,1054.000000,92.500000,497.500000,424.500000,328.500000,322.500000,192.000000,13.000000,750.000000
max,687.000000,238.000000,40.000000,130.000000,121.000000,105.000000,24.000000,14053.000000,4256.000000,548.000000,2165.000000,1659.000000,1566.000000,1377.000000,492.000000,32.000000,2460.000000


Python doesn't have best subset selection so we need to write our own.
We first encode categorical variables. Each categorical features have two levels, so we leave one binary for each, to avoid collinearity.
We will be evauting our model results based on RSS score.

In [45]:
dummies = pd.get_dummies(hitters[['League', 'Division', 'NewLeague']], drop_first=True)
display(dummies.head())

hitters.drop(labels=['League', 'Division', 'NewLeague'], axis="columns", inplace=True)
hitters[['League', 'Division', 'NewLeague']] = dummies

hitters.head()

,League,Division_W,NewLeague_N
-Alan Ashby,1.0,1,1
-Alvin Davis,0.0,1,0
-Andre Dawson,1.0,0,1
-Andres Galarraga,1.0,0,1
-Alfredo Griffin,0.0,1,0


,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary,League,Division,NewLeague
-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,475.0,1.0,1,1
-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,480.0,0.0,1,0
-Andre Dawson,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,500.0,1.0,0,1
-Andres Galarraga,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,91.5,1.0,0,1
-Alfredo Griffin,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25,750.0,0.0,1,0


In [74]:
from itertools import combinations
import statsmodels.api as sm

def best_subset_for_size(size, dataframe, response):
    """
    Returns the best odel from all models with k-predictors (lowest RSS).
    """
    results = {}
    features = dataframe.drop(response, axis = 1).columns
    subsets = list(combinations(features, size))
    for subset in subsets:
        subset = list(subset)
        score, model = calculate_model(subset, response, dataframe)
        if score in results.keys():
            results[score].append(model)
        else:
            results[score] = [model] 

    return results[min(results.keys())]

def calculate_model(subset, response, dataframe):
    """
    Constructs Linear Model Regression of response onto feature and calculates RSSs.
    """
    X = sm.add_constant(dataframe[subset])
    y = dataframe[response]

    model = sm.OLS(y,X).fit()
    score = model.ssr
    return score, model

def best_subset_selection(dataframe, response):
    """
    Does best subset selection.
    """
    best_dict = {}
    for size in range(1,dataframe.shape[1]):
        result = best_subset_for_size(size, dataframe, response)
        best_dict[size] = result

    return best_dict

best_dict = best_subset_selection(dataframe = hitters, response ="Salary")


We now build a dataframe with our model results: